In [ ]:
!pip install PyGithub

In [ ]:
from github import Github
from github import Auth
import pickle
import time
import calendar

In [ ]:
token = '' #Generate personal token under Developer Settings

In [ ]:
auth = Auth.Token(token)
g = Github(auth=auth)

In [ ]:
n = 0

# Enter your query. Note: No matter which way the results are sorted onlyt the first 1000 repositories matching the query are returned.
query = ''  

listofrepos = []
for repo in g.search_repositories(query):  
        
        if g.get_rate_limit().core.remaining > 0:
            repo_dict = {
                'id': repo.id,
                'fullname': repo.full_name,
                'name': repo.name,
                'owner': repo.owner,
                'website': repo.homepage,
                'forks': repo.forks_count,
                'createdAt': repo.created_at,
                'updatedAt': repo.updated_at,
                'commits': repo.get_stats_participation().all,
                'topics': repo.get_topics(),
                'watchers': [watcher.login for watcher in repo.get_subscribers()], #get subscribers fetch people who are watching a repo
                'stargazers': [stargazer.login for stargazer in repo.get_stargazers()], # watchers , watchers_count , and stargazers_count all refer to the same, which is people who have starred a repo
                'contributors': [contributor.login for contributor in repo.get_contributors()]
            } 
            
            try:
                repo_dict['readme'] = repo.get_readme().decoded_content
            except:
                print(f"Failed to collect readme for {repo.full_name}")
            listofrepos.append(repo_dict)
            n += 1
            print(f"{repo.full_name} done. {n} repositories collected. {g.get_rate_limit().core.remaining} calls remaining")
            if n % 100 == 0:
                pickle.dump(listofrepos, open('repositories.pkl', 'wb'))
        
        else:
            core_rate_limit = g.get_rate_limit().core
            reset_timestamp = calendar.timegm(core_rate_limit.reset.timetuple())
            sleep_time = reset_timestamp - calendar.timegm(time.gmtime()) + 5
            print(f'Pickling data before sleeping for {sleep_time} seconds...')
            pickle.dump(listofrepos, open('repositories.pkl', 'wb'))
            time.sleep(sleep_time)

pickle.dump(listofrepos, open('repositories_final.pkl', 'wb'))